In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import string

def process_md_lines(lines):
    lines.append('\n')
    ids = []
    urls = []
    labels = []
    texts = []
    orders = []
    total_lines = []
    id = -1
    url = ''
    order = 0
    for line in lines:
        if line[0] == '#':
            id = int(line[line.index('[') + 1: line.index(']')])
            url = line[line.index(']') + 1:]
            order = 0
        elif line != '\n':
            tokens = line.split('\t')
            labels.append(tokens[0])
            text = tokens[1].lower().translate(str.maketrans('', '', string.punctuation))
            texts.append(text)
            orders.append(order)
            order = order + 1
            ids.append(id)
            urls.append(url)
        elif line == '\n':
            for i in range(order):
                total_lines.append(order)
            order = 0
            id = -1
            url = ''
    df = pd.DataFrame(
        {'id': ids, 'text': texts, 'order': orders, 'total_lines': total_lines, 'url': urls, 'label': labels})
    return df

In [2]:
import pandas as pd
import string
training_lines = open('/kaggle/input/sequentialtext/training.txt','r',encoding='utf-8').readlines()
testing_lines = open('/kaggle/input/sequentialtext/testing.txt','r',encoding='utf-8').readlines()
validation_lines = open('/kaggle/input/sequentialtext/validation.txt','r',encoding='utf-8').readlines()
#lines = open('drive/My Drive/hamdan/data.txt','r',encoding='utf=8').readlines()
training_df = process_md_lines(training_lines)
testing_df = process_md_lines(testing_lines)
validation_df = process_md_lines(validation_lines)
training_df.head()

,id,text,order,total_lines,url,label
0,290,الشريان الأبهر هو أكبر شريان في الجسم وهو ينقل...,0,89,https://www.mayoclinic.org/ar/diseases-conditi...,Overview
1,290,بشكل عام يكون تضيق الأبهر موجودا لدى المصاب من...,1,89,https://www.mayoclinic.org/ar/diseases-conditi...,Overview
2,290,وغالبا ما يصاحب تضيق الأبهر عيوب أخرى في القلب...,2,89,https://www.mayoclinic.org/ar/diseases-conditi...,Overview
3,290,تعتمد أعراض تضيق الأبهر على شدة الحالة فمعظم ا...,3,89,https://www.mayoclinic.org/ar/diseases-conditi...,Symptoms
4,290,قد يبدأ ظهور الأعراض على الأطفال المصابين بتضي...,4,89,https://www.mayoclinic.org/ar/diseases-conditi...,Symptoms


In [3]:
!pip -q install optuna==2.3.0
!pip -q install farasapy
!pip -q install pyarabic
!git clone https://github.com/aub-mind/arabert
    

Cloning into 'arabert'...
remote: Enumerating objects: 600, done.
remote: Counting objects: 100% (65/65), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 600 (delta 38), reused 45 (delta 30), pack-reused 535
Receiving objects: 100% (600/600), 9.14 MiB | 19.34 MiB/s, done.
Resolving deltas: 100% (339/339), done.


In [4]:
import numpy as np

from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import train_test_split
training_df['label'].unique()

array(['Overview', 'Symptoms', 'Causes', 'Diagnosis', 'Treatment'],
      dtype=object)

In [5]:
hard_map = { 0 : 'Overview', 1: 'Causes', 2 : 'Symptoms', 3: 'Treatment', 4:'Diagnosis'}
label_list_HARD = ['Overview', 'Causes', 'Symptoms', 'Treatment', 'Diagnosis']

In [6]:
from arabert.preprocess import ArabertPreprocessor
import numpy as np
from sklearn.metrics import classification_report, accuracy_score, f1_score, confusion_matrix, precision_score , recall_score

from transformers import AutoConfig, AutoModelForSequenceClassification, AutoTokenizer, BertTokenizer
from transformers.data.processors import SingleSentenceClassificationProcessor
from transformers import Trainer , TrainingArguments
from transformers.trainer_utils import EvaluationStrategy
from transformers.data.processors.utils import InputFeatures


In [7]:
# https://huggingface.co/aubmindlab/bert-base-arabertv02
dataset_name = 'medical'
model_name = 'aubmindlab/bert-base-arabertv02'
task_name = 'classification'
max_len = 256
arabert_prep = ArabertPreprocessor(model_name.split("/")[-1])

training_df['text'] = training_df['text'].apply(lambda x:   arabert_prep.preprocess(x))
training_df['text']

0         الشريان الأبهر هو أكبر شريان في الجسم وهو ينقل...
1         بشكل عام يكون تضيق الأبهر موجودا لدى المصاب من...
2         وغالبا ما يصاحب تضيق الأبهر عيوب أخرى في القلب...
3         تعتمد أعراض تضيق الأبهر على شدة الحالة فمعظم ا...
4         قد يبدأ ظهور الأعراض على الأطفال المصابين بتضي...
                                ...                        
296720    إذا كنت والد طفل متلعثم في الكلام فستساعدك كثي...
296721    أنصت لطفلك باهتمام حافظ على تواصل العينين حال ...
296722                                   التواصل مع الآخرين
296723    قد يكون من المفيد للأطفال والوالدين والبالغين ...
296724    لمزيد من المعلومات قم بزيارة مواقع الويب الخاص...
Name: text, Length: 296725, dtype: object

In [8]:
testing_df['text'] = testing_df['text'].apply(lambda x:   arabert_prep.preprocess(x))
testing_df['text']

0        هو حالة مرضية تعنى وجود دم فى البول و يمكن رؤي...
1        إلا أنه فى الغالب لا يمكن إثبات ذلك إلا من خلا...
2        قد لا تكون هناك أى أعراض يشتكى منها المريض سوى...
3                         صعوبة فى التبول مع الشعور بحرقان
4                               ارتفاع فى درجة حرارة الجسم
                               ...                        
36518    الحفاظ على وزن صحي إذا كان وزنك زائدا فإن إنقا...
36519    زيادة الأنشطة البدنية قد تساعد ممارسة الأنشطة ...
36520    الامتناع عن تناول المشروبات الكحولية يمكن للكح...
36521    الامتناع عن التدخين يمكن أن يصيب التبغ جدران ا...
36522    السيطرة على التوتر احرص على تقليل التوتر قدر ا...
Name: text, Length: 36523, dtype: object

In [9]:
X = training_df['text']
y = training_df['label']

In [10]:
X_train, X_test, y_train, y_test =X,testing_df['text'], y,testing_df['label'] #train_test_split(X, y, test_size=0.3)

In [11]:
class BERTDataset():
    def __init__(self, text, target, model_name, max_len, label_map):
      super(BERTDataset).__init__()
      self.text = text
      self.target = target
      self.tokenizer_name = model_name
      self.tokenizer = AutoTokenizer.from_pretrained(model_name)
      self.max_len = max_len
      self.label_map = label_map
      

    def __len__(self):
      return len(self.text)

    def __getitem__(self,item):
      text = str(self.text[item])
      text = " ".join(text.split())


        
      input_ids = self.tokenizer.encode(
          text,
          add_special_tokens=True,
          max_length=self.max_len,
          truncation='longest_first'
      )     
    
      attention_mask = [1] * len(input_ids)

      # Zero-pad up to the sequence length.
      padding_length = self.max_len - len(input_ids)
      input_ids = input_ids + ([self.tokenizer.pad_token_id] * padding_length)
      attention_mask = attention_mask + ([0] * padding_length)    
      
      return InputFeatures(input_ids=input_ids, attention_mask=attention_mask, label=self.label_map[self.target[item]])

In [12]:
label_map = { v:index for index, v in enumerate(label_list_HARD) }
print(label_map)
train_dataset = BERTDataset(X_train.to_list(),y_train.to_list(),model_name, max_len,label_map)
val_dataset  = BERTDataset(X_test.to_list()   ,y_test.to_list()  ,model_name,  max_len, label_map)

{'Overview': 0, 'Causes': 1, 'Symptoms': 2, 'Treatment': 3, 'Diagnosis': 4}


Downloading:   0%|          | 0.00/381 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/384 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/805k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.52M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [13]:
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model_name, return_dict=True, num_labels=len(label_map))

In [14]:
def compute_metrics(p): #p should be of type EvalPrediction
  preds = np.argmax(p.predictions, axis=1)
  assert len(preds) == len(p.label_ids)
  print(classification_report(p.label_ids,preds))
  print(confusion_matrix(p.label_ids,preds))

  # macro_f1_pos_neg = f1_score(p.label_ids,preds,average='macro',labels=[0,1])
  macro_f1 = f1_score(p.label_ids,preds,average='macro')
  macro_precision = precision_score(p.label_ids,preds,average='macro')
  macro_recall = recall_score(p.label_ids,preds,average='macro')
  acc = accuracy_score(p.label_ids,preds)
  # c_repoert= classification_report(p.label_ids,preds))

  return {
      'macro_f1' : macro_f1,
      # 'macro_f1_pos_neg' : macro_f1_pos_neg,  
      'macro_precision': macro_precision,
      'macro_recall': macro_recall,
      'accuracy': acc,
      # 'classification_report' :c_repoert
  }

# HyperParameter Search

In [15]:
training_args = TrainingArguments("./train")
training_args.evaluate_during_training = True
training_args.adam_epsilon = 1e-8
training_args.lr_scheduler_type = 'cosine'
training_args.fp16 = True
training_args.per_device_train_batch_size = 16
training_args.per_device_eval_batch_size = 16
training_args.gradient_accumulation_steps = 2
training_args.num_train_epochs= 8
training_args.evaluation_strategy = EvaluationStrategy.EPOCH
# training_args.logging_steps = 200
training_args.save_steps = 100000
# training_args.save_steps = 
#training_args.eval_steps = 
training_args.disable_tqdm = True
# print("Logging Step:", training_args.logging_steps)
# print("Eval Step:",training_args.eval_steps)

In [16]:
steps_per_epoch = (len(X_train)// (training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps))
total_steps = steps_per_epoch * training_args.num_train_epochs
print(steps_per_epoch)
print(total_steps)

9272
74176


In [17]:
trainer = Trainer(
    args=training_args,
    train_dataset=train_dataset, 
    eval_dataset=val_dataset, 
    model_init=model_init,
    compute_metrics=compute_metrics,
)

loading configuration file https://huggingface.co/aubmindlab/bert-base-arabertv02/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/411eec8d9e12bf4c11eebebb4c5fecd46da787616f45bcfd6cb187e0917afae0.2f0d0092105af7b8b42b899ffb7f801dc48e93516d509483f6cfbd86155d49ea
Model config BertConfig {
  "_name_or_path": "aubmindlab/bert-base-arabertv02",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_to

Downloading:   0%|          | 0.00/518M [00:00<?, ?B/s]

storing https://huggingface.co/aubmindlab/bert-base-arabertv02/resolve/main/pytorch_model.bin in cache at /root/.cache/huggingface/transformers/c0183275aaa055648fb44d6c24d9604b860b02398c7d9c7bea64257cdb87a56e.9e74f71b2acf9ee9289bd6ab56938934df2792f595150523e3f83558666a9676
creating metadata file for /root/.cache/huggingface/transformers/c0183275aaa055648fb44d6c24d9604b860b02398c7d9c7bea64257cdb87a56e.9e74f71b2acf9ee9289bd6ab56938934df2792f595150523e3f83558666a9676
loading weights file https://huggingface.co/aubmindlab/bert-base-arabertv02/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/c0183275aaa055648fb44d6c24d9604b860b02398c7d9c7bea64257cdb87a56e.9e74f71b2acf9ee9289bd6ab56938934df2792f595150523e3f83558666a9676
Some weights of the model checkpoint at aubmindlab/bert-base-arabertv02 were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.d

> # Regular Training

In [18]:
training_args = TrainingArguments("./train")
training_args.evaluate_during_training = True
training_args.adam_epsilon = 1e-8
training_args.learning_rate = 1e-4
training_args.fp16 = True
# training_args.per_device_train_batch_size =16
# training_args.per_device_eval_batch_size = 16

training_args.per_device_train_batch_size = 16  
training_args.per_device_eval_batch_size = 16


training_args.gradient_accumulation_steps = 2
training_args.num_train_epochs= 5


steps_per_epoch = (len(X_train)// (training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps))
total_steps = steps_per_epoch * training_args.num_train_epochs
print(steps_per_epoch)
print(total_steps)
#Warmup_ratio
warmup_ratio = 0.1
training_args.warmup_steps = total_steps*warmup_ratio # or you can set the warmup steps directly 

# training_args.load_best_model_at_end=True

training_args.evaluation_strategy = EvaluationStrategy.EPOCH
# training_args.logging_steps = 200
training_args.save_steps = 100000 #don't want to save any model, there is probably a better way to do this :)
training_args.seed = 42
training_args.disable_tqdm = False
training_args.lr_scheduler_type = 'cosine'

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


9272
46360


In [19]:
trainer = Trainer(
    model = model_init(),
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
) 

loading configuration file https://huggingface.co/aubmindlab/bert-base-arabertv02/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/411eec8d9e12bf4c11eebebb4c5fecd46da787616f45bcfd6cb187e0917afae0.2f0d0092105af7b8b42b899ffb7f801dc48e93516d509483f6cfbd86155d49ea
Model config BertConfig {
  "_name_or_path": "aubmindlab/bert-base-arabertv02",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_to

In [20]:
trainer.train()

/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 296725
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 2
  Total optimization steps = 23180
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=23180, training_loss=0.22259599436347705, metrics={'train_runtime': 16827.6458, 'train_samples_per_second': 88.166, 'train_steps_per_second': 1.377, 'total_flos': 1.9518156431608013e+17, 'train_loss': 0.22259599436347705, 'epoch': 5.0})

In [21]:
trainer.evaluate(val_dataset)

***** Running Evaluation *****
  Num examples = 36523
  Batch size = 32


              precision    recall  f1-score   support

           0       0.97      0.97      0.97      3572
           1       0.96      0.95      0.96      6114
           2       0.95      0.97      0.96      7663
           3       0.99      0.98      0.99     13022
           4       0.99      0.98      0.98      6152

    accuracy                           0.97     36523
   macro avg       0.97      0.97      0.97     36523
weighted avg       0.97      0.97      0.97     36523

[[ 3454    58    31    17    12]
 [   49  5830   173    57     5]
 [   31   110  7461    34    27]
 [   22    71    89 12821    19]
 [   14    24    78    27  6009]]


{'eval_loss': 0.09704025834798813,
 'eval_macro_f1': 0.9711319854833025,
 'eval_macro_precision': 0.9712355148194298,
 'eval_macro_recall': 0.971094838649495,
 'eval_accuracy': 0.9740437532513758,
 'eval_runtime': 154.1768,
 'eval_samples_per_second': 236.89,
 'eval_steps_per_second': 7.407,
 'epoch': 5.0}

In [22]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
trainer._save("/kaggle/working/AraBERTV02_model")
tokenizer.save_pretrained("/kaggle/working/AraBERTV02_tokenizer")

loading configuration file https://huggingface.co/aubmindlab/bert-base-arabertv02/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/411eec8d9e12bf4c11eebebb4c5fecd46da787616f45bcfd6cb187e0917afae0.2f0d0092105af7b8b42b899ffb7f801dc48e93516d509483f6cfbd86155d49ea
Model config BertConfig {
  "_name_or_path": "aubmindlab/bert-base-arabertv02",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.20.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 64000
}

loading file https://huggingface.co/aubmindlab/bert-base-arabertv02/res

('/kaggle/working/AraBERTV02_tokenizer/tokenizer_config.json',
 '/kaggle/working/AraBERTV02_tokenizer/special_tokens_map.json',
 '/kaggle/working/AraBERTV02_tokenizer/vocab.txt',
 '/kaggle/working/AraBERTV02_tokenizer/added_tokens.json',
 '/kaggle/working/AraBERTV02_tokenizer/tokenizer.json')